Import the Q# module.

This enables the `%%qsharp` magic and initializes a Q# interpreter singleton.

In [ ]:
import qsharp

Run Q# using the `%%qsharp` magic.

`DumpMachine()` and `Message()` output get formatted as HTML. Return value is shown as cell output.

In [ ]:
%%qsharp

operation Main() : Result {
    use q = Qubit();
    X(q);
    Microsoft.Quantum.Diagnostics.DumpMachine();
    let r = M(q);
    Message($"The result of the measurement is {r}");
    Reset(q);
    r
}

Main()

`qsharp.eval()` does the same thing as the `%%qsharp` magic.

`DumpMachine()` and `Message()` print to stdout and get displayed in the notebook as plain text

In [ ]:
qsharp.eval("Main()")

Assign a result to a Python variable.

In [ ]:
result = qsharp.eval("1 + 2")

print(f"Result: {result} (type: {type(result).__name__})")

Errors are exceptions. 

Catch and handle compilation errors.

In [ ]:
from qsharp import QSharpError

try:
    qsharp.eval(
        """
operation Foo() : Unit {
    Bar();
    Baz();
}
"""
    )
except QSharpError as ex:
    print(ex)

Catch and handle runtime errors.

In [ ]:
try:
    qsharp.eval("operation Foo() : Unit { use q = Qubit(); X(q) } Foo()")
except QSharpError as ex:
    print(ex)

A runtime error that's not caught gets reported as a Python exception.

In [ ]:
qsharp.eval("operation Foo() : Unit { use q = Qubit(); X(q) } Foo()")

In `%%qsharp` cells, exceptions are handled and displayed as error text.

In [ ]:
%%qsharp

operation Bar() : Unit {
    use q = Qubit(); 
    Microsoft.Quantum.Diagnostics.DumpMachine(); 
    X(q);
} 
    
Bar()

Streaming output for long running operations.

In [ ]:
%%qsharp

open Microsoft.Quantum.Diagnostics;

operation Main() : Unit {
    Message("Generating random bit... ");
    for i in 0..400000 {
        use q = Qubit();
        H(q);
        let r = M(q);
        if (i % 100000) == 0 {
            DumpMachine();
            Message($"Result: {r}");
        }
        Reset(q);
    }
}

Main()